## Install dependencies

In [82]:
!pip install essentia

In [83]:
!pip install mir_eval

In [ ]:
!pip install mido

In [ ]:
!pip install pretty_midi

## Import

In [ ]:
import librosa
import essentia.standard as es
import mido
import pretty_midi
import mir_eval.display
from mir_eval.sonify import pitch_contour



import numpy as np
import matplotlib.pyplot as plt
import IPython

In [ ]:
%matplotlib inline

## Import Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# file path
path = 'drive/MyDrive/Colab Notebooks/audios/'
fileName = 'Pop dance track with catchy melodies'
ext = '.mp3'

Audio

## Essentia to pretty midi conversion

In [ ]:
# Recommended: Apply equal-loudness filter for PredominantPitchMelodia.
loader = es.EqloudLoader(filename=path+fileName+ext, sampleRate=44100)
audio = loader()
print(f"Duration of audio sample [sec]: {len(audio)/44100.0}")

In [ ]:
# PitchMelodia takes  entire audio signal as input.
pitch_extractor = es.PredominantPitchMelodia(frameSize=2048, hopSize=128, guessUnvoiced=True)
pitch_values, pitch_confidence = pitch_extractor(audio)

# Pitch estimated on frames -> compute frame time positions.
pitch_times = np.linspace(0.0,len(audio)/44100.0,len(pitch_values))

In [ ]:
tmpAudio = path + 'results/' + 'syn_' + fileName + ext

# Essentia operates with float32 ndarrays instead of float64, so let's cast it.
synthesized_melody = pitch_contour(pitch_times, pitch_values, 44100).astype(np.float32)[:len(audio)]
es.AudioWriter(filename=tmpAudio, format='mp3')(es.StereoMuxer()(audio, synthesized_melody))

In [ ]:
onsets, durations, notes = es.PitchContourSegmentation(hopSize=128)(pitch_values, audio)
print("MIDI notes:", notes) # Midi pitch number
print("MIDI note onsets:", onsets)
print("MIDI note durations:", durations)

## Piano Roll

In [ ]:
# Piano roll (time-frequency matrix)
def plot_piano_roll(pm, start_pitch, end_pitch, fs=100):
    # Use librosa's specshow function for displaying the piano roll
    librosa.display.specshow(pm.get_piano_roll(fs)[start_pitch:end_pitch],
                             hop_length=1, sr=fs, x_axis='time', y_axis='cqt_note',
                             fmin=pretty_midi.note_number_to_hz(start_pitch))

In [ ]:
pm = pretty_midi.PrettyMIDI(midi_file)

In [ ]:
plt.figure(figsize=(12, 4))
plot_piano_roll(pm, 58, 84, fs=400)

In [ ]:
# Let's look at what's in this MIDI file
print('There are {} time signature changes'.format(len(pm.time_signature_changes)))
print('There are {} instruments'.format(len(pm.instruments)))
print('Instrument 3 has {} notes'.format(len(pm.instruments[0].notes)))
#print('Instrument 4 has {} pitch bends'.format(len(pm.instruments[4].pitch_bends)))
#print('Instrument 5 has {} control changes'.format(len(pm.instruments[5].control_changes)))

## Analyizing MIDI and Essentia data

In [ ]:
# Plot a pitch class distribution - sort of a proxy for key
pm = pretty_midi.PrettyMIDI(midi_file)
plt.bar(np.arange(12), pm.get_pitch_class_histogram());
plt.xticks(np.arange(12), ['C', '', 'D', '', 'E', 'F', '', 'G', '', 'A', '', 'B'])
plt.xlabel('Note')
plt.ylabel('Proportion')

pitch_class_histogram = pm.get_pitch_class_histogram()
print("pitch class histogram percentage values:",pitch_class_histogram)



#calculate absolute pitch class values
pitch_extractor = es.PredominantPitchMelodia()
pitch_values, pitch_confidence = pitch_extractor(audio)

empthy_frames = list(filter(lambda x: x == 0, pitch_values))
print("Number of emthy frames:",len(empthy_frames))

print("number of all pitch values:", len(pitch_values))

print("Ratio of emthy frames:",len(empthy_frames)/len(pitch_values))

pitchesE = list(filter(lambda x: x != 0, pitch_values))
print("All non cero pitch values: ", pitchesE)
print("Number of non-cero pitch values: ", len(pitchesE))


#absoluteClassValues = [i *  len(pitchesE) for i in pitch_class_histogram]
absoluteClassValues = [round(i * len(pitchesE)) for i in pitch_class_histogram]
print(absoluteClassValues)

pitch_class_histogram = pm.get_pitch_class_histogram()
pitch_class_histogram = [round(i,2) for i in pitch_class_histogram]
print(pitch_class_histogram)


